In [ ]:
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import resample
import pickle
import pandas as pd
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
import os  

In [ ]:
def read_pickle(file_dir):   
    pickle_L=[]   
    pickle_name=[]
    for dirpath, dirnames, filenames in os.walk(file_dir):  
        for file in filenames :  
            if os.path.splitext(file)[1] == '.pickle':  
                pickle_L.append(os.path.join(dirpath, file))  
                pickle_name.append(os.path.join(file))  
            
    #return L 
    return  pickle_name
    #return name    

In [ ]:
def pickleLoader(pklFile):
    try:
        while True:
            yield pickle.load(pklFile)
    except EOFError:
        pass

In [ ]:
tra_path='Epi-AI_processed_files/Train/Model_I/'

In [ ]:
train_file=read_pickle(tra_path)

In [ ]:
train_frame1=[]
for i in range(0,len(train_file)):
    with open(tra_path+train_file[i],'rb') as file:
        #print(file)
        for event in pickleLoader(file):
            train_frame1.append(event)

train_re1= pd.concat(train_frame1, ignore_index=True)       

In [ ]:
train_com1=train_re1[['target','mean_EEG2', 'EEG2_mobility', 'TKEO_EEG2', 'EEG2_P_delta','EEG2_P_theta',
                                'EEG2_P_alpha','EEG2_P_beta','EEG2_P_gamma','EEG2_P_total','EEG2_rel_delta','EEG2_rel_theta',
                                'EEG2_rel_alpha','EEG2_rel_beta','EEG2_rel_gamma','EEG2_pfd','EEG2_skew',
                                'EEG2_kurtosis','EEG2_var','EEG2_envelope']]

In [ ]:
tra_path2='Epi-AI_processed_files/Train/Model_II/'
train_file2=read_pickle(tra_path2)

In [ ]:
train_frame2=[]
for i in range(0,len(train_file2)):
    with open(tra_path2+train_file2[i],'rb') as file2:
        #print(file)
        for event2 in pickleLoader(file2):
            train_frame2.append(event2)
train_re2= pd.concat(train_frame2, ignore_index=True)

In [ ]:
train_com2=train_re2[['target','mean_EEG2', 'EEG2_mobility', 'TKEO_EEG2', 'EEG2_P_delta','EEG2_P_theta',
                                'EEG2_P_alpha','EEG2_P_beta','EEG2_P_gamma','EEG2_P_total','EEG2_rel_delta','EEG2_rel_theta',
                                'EEG2_rel_alpha','EEG2_rel_beta','EEG2_rel_gamma','EEG2_pfd','EEG2_skew',
                                'EEG2_kurtosis','EEG2_var','EEG2_envelope']]

In [ ]:
tra_path3='Epi-AI_processed_files/Train/Model_III/'
train_file3=read_pickle(tra_path3)

In [ ]:
train_frame3=[]
for i in range(0,len(train_file3)):
    with open(tra_path3+train_file3[i],'rb') as file3:
        #print(file)
        for event3 in pickleLoader(file3):
            train_frame3.append(event3)
train_re3= pd.concat(train_frame3, ignore_index=True)

In [ ]:
train_com3=train_re3[['target','mean_EEG2', 'EEG2_mobility', 'TKEO_EEG2', 'EEG2_P_delta','EEG2_P_theta',
                                'EEG2_P_alpha','EEG2_P_beta','EEG2_P_gamma','EEG2_P_total','EEG2_rel_delta','EEG2_rel_theta',
                                'EEG2_rel_alpha','EEG2_rel_beta','EEG2_rel_gamma','EEG2_pfd','EEG2_skew',
                                'EEG2_kurtosis','EEG2_var','EEG2_envelope']]

In [ ]:
frames = [train_com1, train_com2, train_com3]

data_train = pd.concat(frames)

In [ ]:
y = data_train.target
X = data_train.drop('target', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X ,y, test_size=0.2, random_state=0)

# Model

In [ ]:
'''# xgboost for feature importance on a classification problem
from xgboost import XGBClassifier
from matplotlib import pyplot
# define the model
model = XGBClassifier(
     learning_rate =0.01,
     n_estimators=100,
     max_depth=6,
     scale_pos_weight=478, 
     random_state=2)
# fit the model
model.fit(X_train, y_train)
'''

In [ ]:
# save the model to disk
#from sklearn.externals import joblib
#filename = 'JNE_XG_W478.sav'
#joblib.dump(model, filename)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc
threshold=0.5

In [ ]:
import joblib
#load model
filename='JNE_XG_W478.sav'

model=joblib.load(filename)

# performance on the training set

In [ ]:
y_tra = y_train
X_tra = X_train
#pred_y_1 =model.predict(X_1)
predicted_proba = model.predict_proba(X_tra)
pred_y_tra = (predicted_proba [:,1] >= threshold).astype('int')

print("Accuracy: ", metrics.accuracy_score(y_tra, pred_y_tra))
print("ROC : ", roc_auc_score(y_tra,pred_y_tra))
print("Confusion matrix: \n", metrics.confusion_matrix(y_tra,pred_y_tra))
print("Classification report:\n ", metrics.classification_report(y_tra, pred_y_tra,digits=4))

tn, fp, fn, tp = confusion_matrix(y_tra,pred_y_tra).ravel()
specificity = tn / (tn+fp)
sens=tp/(tp+fn)

print('Sens: ', sens )
print('Specificity:', specificity)

# performance on the validation set

In [ ]:
X_test.columns=['mean', 'mobility','TKEO', 'absolute delta power','absolute theta power',
               'absolute alpha power','absolute beta power','absolute gamma power','absolute total power',
               'relative delta power','relative theta power','relative alpha power','relative beta power',
               'relative gamma power','fractal dimension','skewness',
                'kurtosis','variance','signal envelope']

In [ ]:
y_t = y_test
X_t = X_test
#pred_y_1 =model.predict(X_1)
predicted_proba = model.predict_proba(X_t)
pred_y_t = (predicted_proba [:,1] >= threshold).astype('int')

print("Accuracy: ", metrics.accuracy_score(y_t, pred_y_t))
print("ROC : ", roc_auc_score(y_t,pred_y_t)) 
print("Confusion matrix: \n", metrics.confusion_matrix(y_t,pred_y_t))
print("Classification report:\n ", metrics.classification_report(y_t, pred_y_t,digits=4))

tn, fp, fn, tp = confusion_matrix(y_t,pred_y_t).ravel()
specificity = tn / (tn+fp)
sens=tp/(tp+fn)

print('Sens: ', sens )
print('Specificity:', specificity)